In [1]:
!pip install -qU crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
from utils import get_bedrock_client, get_haiku_config, get_sonnet_config
from langchain_community.chat_models import BedrockChat  # Changed this import
from crewai import Agent, Task, Crew

In [4]:
# import os
# from utils import get_openai_api_key

# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'


bedrock_client = get_bedrock_client()
sonnet_config = get_sonnet_config()  # Using Sonnet config

# Create BedrockChat instance for CrewAI
llm = BedrockChat(
    client=bedrock_client,
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # Updated model ID
    streaming=True,
    model_kwargs={
        "temperature": sonnet_config["temperature"],
        "top_p": sonnet_config["top_p"],
        "max_tokens": sonnet_config["max_tokens"],
        "anthropic_version": "bedrock-2023-05-31"
    }
)

# Set model name in environment if needed
os.environ["MODEL_NAME"] = "anthropic.claude-3-sonnet-20240229-v1:0"

## crewAI Tools

In [23]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')

- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [24]:
from IPython.display import Markdown, display
display(Markdown("./fake_resume.md"))


# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a distinguished Software Engineering Leader with an 18-year tenure in the technology industry, where he has excelled in leading both remote and in-office engineering teams. His expertise spans across software development, process innovation, and enhancing team collaboration. He is highly proficient in programming languages such as Ruby, Python, JavaScript, TypeScript, and Elixir, alongside deep expertise in various front end frameworks. Noah's significant experience in data science and machine learning has enabled him to spearhead successful deployments of scalable AI solutions and innovative data model development.

## Work History

### DataKernel: Director of Software Engineering (remote) — 2022 - Present
- Noah has transformed the engineering division into a key revenue pillar for DataKernel, rapidly expanding the customer base from inception to a robust community.
- He spearheaded the integration of cutting-edge AI technologies and the use of scalable vector databases, which significantly enhanced the product's capabilities and market positioning.
- Under his leadership, the team has seen a substantial growth in skill development, with a focus on achieving strategic project goals that have significantly influenced the company's direction.
- Noah also played a critical role in defining the company’s long-term strategic initiatives, particularly in adopting AI technologies that have set new benchmarks within the industry.

### DataKernel: Senior Software Engineering Manager (remote) — 2019 - 2022
- Directed the engineering strategy and operations in close collaboration with C-level executives, playing a pivotal role in shaping the company's technological trajectory.
- Managed diverse teams across multiple time zones in North America and Europe, creating an environment of transparency and mutual respect which enhanced team performance and morale.
- His initiatives in recruiting, mentoring, and retaining top talent have been crucial in fostering a culture of continuous improvement and high performance.

### InnovPet: Founder & CEO (remote) — 2019 - 2022
- Noah founded InnovPet, a startup focused on innovative IoT solutions for pet care, including a revolutionary GPS tracking collar that significantly enhanced pet safety and owner peace of mind.
- He was responsible for overseeing product development from concept through execution, working closely with engineering teams and marketing partners to ensure a successful market entry.
- Successfully set up an advisory board, established production facilities overseas, and navigated the company through a successful initial funding phase, showcasing his leadership and entrepreneurial acumen.
- Built the initial version of the product leveraging MongoDB

### EliteDevs: Engineering Manager (remote) — 2018 - 2019
- Noah was instrumental in formulating and executing strategic plans that enhanced inter-departmental coordination and trust, leading to better project outcomes.
- He managed multiple engineering teams, fostering a culture that balances productivity with innovation, and implemented goal-setting frameworks that aligned with the company's long-term goals.
- Was a bery hands on manager using ruby on rails and react to build out a new product.

### PrintPack: Engineering Manager (remote) — 2016 - 2018
- Led the formation and development of a high-performance engineering team that was pivotal in increasing company revenue by 500% within two years.
- His leadership in integrating data analytics into business decision-making processes led to the development of a predictive modeling tool that revolutionized customer behavior analysis.

### DriveAI: Senior Software Engineer (remote) — 2015 - 2016
- Developed and optimized a central API that significantly improved the functionality used by a large engineering team and thousands of users, enhancing overall system performance and user satisfaction.
- Implemented several critical enhancements, including advanced caching strategies that drastically reduced response times and system loads.

### BetCraft: CTO — 2013 - 2015
- Led the technology strategy post-Series A funding, directly reporting to the board and guiding the company through a phase of significant technological advancement and network expansion.
- His strategic initiatives and partnerships significantly improved platform performance and expanded the company's market reach.
- Helped build his initial product using both React and Angular and got pretty good at it.

## Education

### MBA in Information Technology
London Business School - MBA

### Advanced Leadership Techniques
University of London - Certification

### Data Science Specialization
Coursera (Johns Hopkins University) - Certification

### B.Sc. in Computer Science
University of Edinburgh - Bachelor’s degree

Noah Williams is an ideal candidate for senior executive roles, particularly in companies seeking leadership with a robust blend of technical and strategic expertise.


## Creating Agents

In [25]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    ),
    llm=llm,
    max_iter=5,
    memory=True,
)

In [27]:
# # Configure MDXSearchTool with supported providers
# tool = MDXSearchTool(
#     config=dict(
#         llm=dict(
#             provider="anthropic",  # Anthropic is supported
#             config=dict(
#                 model="claude-3-sonnet-20240229-v1:0",
#                 temperature=sonnet_config["temperature"],
#                 top_p=sonnet_config["top_p"],
#                 max_tokens=sonnet_config["max_tokens"]
#             ),
#         ),
#         embedder=dict(
#             provider="openai",  # Using OpenAI for embeddings since Bedrock isn't supported
#             config=dict(
#                 model="text-embedding-3-small",
#                 task_type="retrieval_document"
#             ),
#         ),
#     )
# )

In [29]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    ),
     llm=llm,
    max_iter=5,
    memory=True,
)

In [30]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    ),
     llm=llm,
    max_iter=5,
    memory=True,
)

In [31]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    ),
     llm=llm,
    max_iter=5,
    memory=True,
)

## Creating Tasks

In [32]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [33]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [34]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [35]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [36]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [37]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [38]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his abili

- Dislplay the generated `tailored_resume.md` file.

In [41]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

João Moura
Software Engineering Leader & Full Stack Developer

Summary:
Accomplished Software Engineering Leader with 18 years of experience in managing remote and in-office teams. Proven expertise in full-stack development, leveraging proficiency in programming languages (Python, Ruby, Elixir), front-end frameworks (React, Angular), and back-end frameworks (Rails, Phoenix). Skilled in AI/ML model development, data science, and innovative technology solutions. Driven by a passion for cutting-edge technologies and a commitment to continuous learning and professional growth.

Technical Skills:
Front-end: HTML, CSS, JavaScript, XML, jQuery, React, Angular
Back-end: Python, Java, Ruby, Node.js, Django, Ruby on Rails, Express.js
Databases: MySQL, MongoDB, SQL, NoSQL
Web Development: RESTful APIs, web architecture, HTTP/HTTPS protocols
AI/ML: Expertise in developing and deploying scalable AI solutions and data models
Other: Git, Docker, AWS, Azure, Agile methodologies, web security best practices

Professional Experience:

Software Engineering Leader, DataKernel (2020 - Present)
- Spearheaded the integration of cutting-edge AI technologies and scalable vector databases, enhancing product capabilities and market positioning.
- Managed cross-functional engineering teams, fostering collaboration and driving process innovation.
- Leveraged expertise in data science and machine learning to develop innovative data models and solutions.

Founder & CEO, InnovPet (2018 - 2020)
- Founded and led the development of an innovative IoT solution for pet care, including a GPS tracking collar, from concept to execution.
- Oversaw the engineering team, utilizing MongoDB and leveraging agile methodologies for efficient product development.
- Implemented robust security measures and data privacy best practices to ensure compliance and customer trust.

Engineering Manager, EliteDevs (2016 - 2018)
- Managed a team of full-stack developers, utilizing Ruby on Rails and React to build out a new product.
- Implemented continuous integration and deployment pipelines, ensuring efficient and reliable software delivery.
- Fostered a culture of collaboration, knowledge sharing, and continuous improvement within the engineering team.

Software Engineer, BetCraft (2014 - 2016)
- Contributed to the development of the initial product, gaining proficiency in React and Angular front-end frameworks.
- Collaborated with cross-functional teams to ensure seamless integration of front-end and back-end components.
- Participated in code reviews and maintained high coding standards, ensuring maintainable and scalable codebase.

Education:
Master's Degree in Computer Science, University of Lisbon (2010 - 2012)
Bachelor's Degree in Computer Engineering, University of Coimbra (2006 - 2010)

Certifications:
AWS Certified Solutions Architect - Associate
Certified Scrum Master (CSM)

Open-Source Contributions:
- crewAI: Framework for orchestrating role-playing, autonomous AI agents
- machinery: Elixir state machine library
- active_model_serializers: Ruby serialization implementation
- gioco: Gamification gem for Ruby on Rails applications
- sigma: Ranking algorithm for Ruby on Rails applications
- keeper: Flexible and simple authentication solution for Phoenix

- Dislplay the generated `interview_materials.md` file.

In [42]:
display(Markdown("./interview_materials.md"))

Potential Interview Questions and Talking Points for João Moura:

Technical Skills:

1. Can you walk us through your experience with front-end development frameworks like React and Angular? How have you leveraged these technologies to create responsive and user-friendly interfaces?

Talking Points:
- Highlight proficiency in React and Angular, including specific projects or features developed using these frameworks.
- Discuss techniques for optimizing user experience across different devices and browsers.
- Showcase understanding of modern front-end development best practices and trends.

2. Can you describe your experience with back-end development frameworks like Ruby on Rails and Phoenix? How have you utilized these frameworks to build scalable and robust server-side applications?

Talking Points:
- Emphasize expertise in Ruby on Rails and Phoenix frameworks, including any notable projects or challenges overcome.
- Discuss strategies for ensuring efficient data flow and seamless integration between front-end and back-end components.
- Highlight experience with RESTful APIs, web architecture, and HTTP/HTTPS protocols.

3. You have experience with AI/ML model development and data science. Can you provide an example of a project where you successfully integrated AI/ML capabilities into an application? What challenges did you face, and how did you overcome them?

Talking Points:
- Describe a specific project involving AI/ML model development and deployment.
- Discuss the challenges faced, such as data preprocessing, model training, and scalability.
- Highlight the innovative solutions and techniques employed to overcome these challenges.

4. Can you discuss your approach to ensuring data privacy and security in your projects? How have you implemented best practices and adhered to relevant regulations?

Talking Points:
- Demonstrate understanding of data privacy regulations and industry best practices.
- Provide examples of security measures implemented in previous projects to protect sensitive data.
- Discuss strategies for maintaining data integrity and user trust.

Project Experience and Problem-Solving:

5. Can you describe a complex project you have worked on and the challenges you faced? How did you approach problem-solving and collaborate with your team to overcome these challenges?

Talking Points:
- Highlight a specific project that involved complex technical challenges or constraints.
- Discuss the problem-solving strategies employed, such as breaking down problems, researching solutions, and seeking input from team members.
- Emphasize collaboration and communication skills in working with cross-functional teams.

6. Can you provide an example of a time when you had to learn a new technology or skill quickly to meet project requirements? How did you approach the learning process, and what strategies did you use to become proficient?

Talking Points:
- Describe a situation where rapid learning was required to meet project demands.
- Discuss the learning strategies employed, such as online resources, documentation, and hands-on practice.
- Highlight the ability to adapt and learn quickly in a fast-paced environment.

Leadership and Cultural Fit:

7. As an experienced Software Engineering Leader, how do you foster a culture of innovation and continuous learning within your teams?

Talking Points:
- Discuss strategies for encouraging knowledge sharing and collaboration among team members.
- Highlight initiatives or practices implemented to promote professional growth and skill development.
- Emphasize the importance of staying up-to-date with emerging technologies and industry trends.

8. Can you describe your approach to managing remote and in-office teams? How do you ensure effective communication and collaboration across distributed teams?

Talking Points:
- Discuss strategies for facilitating communication and collaboration in remote and hybrid team environments.
- Highlight tools and processes implemented to ensure transparency and alignment across teams.
- Emphasize the importance of building trust, setting clear expectations, and providing support to team members.

9. What motivates you as a Software Engineering Leader, and how do you inspire and motivate your teams to achieve their best?

Talking Points:
- Share personal motivations and passion for technology, innovation, and continuous learning.
- Discuss leadership strategies for fostering a positive and supportive team culture.
- Highlight the importance of setting clear goals, providing constructive feedback, and recognizing achievements.

10. Can you describe a situation where you had to navigate a challenging team dynamic or conflict? How did you approach the situation, and what did you learn from the experience?

Talking Points:
- Provide an example of a challenging team dynamic or conflict situation.
- Discuss the strategies employed to address the issue, such as active listening, empathy, and conflict resolution techniques.
- Highlight the lessons learned and how the experience contributed to personal and professional growth.

These potential interview questions and talking points cover various aspects of João Moura's qualifications, including technical skills, project experience, problem-solving abilities, leadership, and cultural fit. By effectively addressing these areas, João can showcase his expertise, highlight his strengths, and demonstrate his suitability for the Full Stack Developer role.